# Stable Video Diffusion (SVD)
Generate a short video from a single input image using SVD.

In [ ]:
!pip install -q diffusers transformers accelerate safetensors decord imageio

In [ ]:
from diffusers import StableVideoDiffusionPipeline
import torch
import decord
from PIL import Image
import imageio
import numpy as np
from google.colab import files

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid",
    torch_dtype=torch.float16,
    variant="fp16"
).to(device)

In [ ]:
# Upload image
uploaded = files.upload()
input_image_path = next(iter(uploaded))
image = Image.open(input_image_path).convert("RGB")
image = image.resize((512, 512))

In [ ]:
# Generate video
video_frames = pipe(image).frames
output_path = "svd_output.mp4"
imageio.mimsave(output_path, [np.array(f) for f in video_frames], fps=7)
files.download(output_path)